In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%run ../nli/config.py
%run ../nli/nli_finetune.py
%run ../nli/mnli.py
%run ../nli/nli_dataset.py

In [3]:
import pandas as pd

import torch
import transformers
import torchmetrics
import pytorch_lightning as pl

from transforms import ClinicalSynonymSubstitution

In [4]:
%%capture
transform=ClinicalSynonymSubstitution(substitution_probability=1.0,p=0.9)

In [5]:
text="Patient has elevated BUN"
transformed_text=transform(text)
transformed_text

/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


'Patients has elevate Blood Urea Nitrogen Measurement'

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
CONFIG["MODEL_NAME_OR_PATH"]="dmis-lab/biobert-v1.1" # Make changes here to specify different model.
CONFIG["CHECKPOINT_DIR"]="../nli/checkpoints/biobert_v1-epoch=01-val_loss=0.45.ckpt" # Make changes here to specify corresponding model checkpoint.

In [8]:
_=pl.seed_everything(CONFIG['SEED'])

Global seed set to 13


In [9]:
trained_model = NLIFineTuningModel.load_from_checkpoint(checkpoint_path=CONFIG["CHECKPOINT_DIR"],  # model checkpoint path
                                                        num_labels=CONFIG['NUM_CLASSES'],
                                                        model_name_or_path=CONFIG['MODEL_NAME_OR_PATH'])
trained_model.freeze()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    CONFIG['MODEL_NAME_OR_PATH'])

In [11]:
test_df=mnli_df('test')

In [12]:
test_dataset = NLIDataset(max_len=CONFIG['MAX_LEN'],
                          tokenizer=tokenizer,
                          sentence1=test_df[CONFIG['sentence1']].values,
                          sentence2=test_df[CONFIG['sentence2']].values,
                          labels=test_df[CONFIG['labels']].values,
                          transforms=transform
                         )

In [13]:
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                                      batch_size=CONFIG['VAL_BS'],
                                                      shuffle=False,
                                                      num_workers=CONFIG['NUM_WORKERS'])

In [14]:
test_metrics=torchmetrics.MetricCollection([
            torchmetrics.Accuracy(),
            torchmetrics.F1(num_classes=CONFIG['NUM_CLASSES'], average='macro')])
test_metrics=test_metrics.to(device)

In [15]:
trained_model=trained_model.to(device)

In [16]:
for batch in test_dataloader:
    for key,value in batch.items():
        batch[key]=value.to(device)
    loss, logits = trained_model(batch)[:2]
    predictions = torch.argmax(logits, dim=1)
    test_metrics(predictions, batch['labels'])
test_metrics.compute()

/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]
/home/va2134/.local/lib/python3.8/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray fro

{'Accuracy': tensor(0.7004, device='cuda:0'),
 'F1': tensor(0.6995, device='cuda:0')}